### ✍️ Required libraries & settings

In [ ]:
import sys
sys.path.insert(1, '/home/sbanik@quansight.com/demo-dashboards')

import warnings
warnings.filterwarnings('ignore')

from intake_utils import (
    catalog_init,
    list_catalog,
    view_catalog)

import numpy as np
import panel as pn
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
from holoviews import opts,dim
from bokeh.models import HoverTool
hv.extension('bokeh')

### 👓 About dataset

- **Title :** Mauna Loa CO2 monthly (mean) data
- **Brief description :** 
Observation taken at Mauna Loa Observatory, Hawaii. The carbon dioxide data on Mauna Loa constitute the longest record of direct measurements of CO2 in the atmosphere. They were started by C. David Keeling of the Scripps Institution of Oceanography in March of 1958 at a facility of the National Oceanic and Atmospheric Administration [Keeling, 1976]. NOAA started its own CO2 measurements in May of 1974, and they have run in parallel with those made by Scripps since then [Thoning, 1989].

#### 🗒 Columns in this Dataset {[info source](https://datahub.io/core/co2-ppm#resource-co2-mm-mlo)}:
- **year** : (YYYY) year of the calender when observation was recorder
- **month** : (MM) calender month when observation was recorder
- **decimal date** : based in decimal calender system (A decimal calendar is a calendar which includes units of time based on the decimal system)
- **average** : The monthly mean CO2 mole fraction determined from daily averages. If there are missing days concentrated either early or late in the month, the monthly mean is corrected to the middle of the month using the average seasonal cycle.
- **deseasonalized**: The interpolated monthly mean is then the sum of the average seasonal cycle value and the trend value for the missing month
- **ndays** : -1 denotes no data for number of daily averages in the month.
- **sdev** : standard deviation
- **unc** : Uncertainity estimated from the standard deviation of the differences between monthly mean values determined independently by the Scripps Institution of Oceanography and by NOAA/ESRL.

### 📖 Reading via from `intake` catalog


In [ ]:
catalog = catalog_init()

### 👓 Data preview and dtype info

In [ ]:
data = catalog.mauna_loa_co2.read()
data.head(5)

In [ ]:
data.info()

### 🔍 Insights and exploration

In [ ]:
group_plot_title = 'Co2 concentration (1996:2020)'
plot_avg = hv.Curve(
    (data.year, data.average),
    "Year",
    "CO2 concentration in ppm", 
    label="Average")
plot_deseasonalized = hv.Curve(
    (data.year, data.deseasonalized),
    "Year", 
    "CO2 concentration in ppm",
    label="Deseasonalized")

hover = HoverTool(tooltips=[('Year','@Year'), ('Co2 (ppm)','$y')])
plot_avg.opts(height=400, width=700, 
              tools=[hover])

layout = (plot_avg * plot_deseasonalized).relabel(group_plot_title)
layout.opts(opts.Layout(
    shared_axes=True,
    merge_tools=True)).opts(
    legend_position = 'right',
    legend_offset=(0, 200))

#### Insights
- Over the years the Co2 concentration has increased significantly

### 🗓 Monthly variation in Co2 levels 

In [ ]:
key_dimensions   = [('year', 'Year'), ('month', 'Month')]
value_dimensions = [('average', 'Average Co2 (ppm)'), ('deseasonalized', 'Deseasonalized Co2 (ppm)')]
mini = hv.Table(data, key_dimensions, value_dimensions)

hover = HoverTool(tooltips=[("year", "@year"),
                            ("Concentration of Co2 (ppm)", "$y")])
plot_1 = mini.to.curve('Year',['Average Co2 (ppm)'], label="Average Co2 concentration in ppm").opts(
    width=650, tools=[hover], line_width=0.75, color='grey')
plot_2 = mini.to.curve('Year',['Deseasonalized Co2 (ppm)'], label="Interpolated Co2 concentration in ppm").opts(
    width=650, tools=[hover], line_width=1.0, color='red', line_dash='dotted')

(plot_1 * plot_2).opts(shared_axes=True, legend_position='bottom_right', legend_offset=(0, 200))

### Insights
- Both estimated and interpolated Co2 concentration levels are close in month of `Jan` and `Dec`
- In month of `May`, `Sep` and `Oct`, the observed difference between estimated and interpolated values is highest 
